In [1]:
import numpy as np
import mne
import os
import matplotlib.pyplot as plt
import seaborn as sns
import glob
from scipy.signal import welch
from sklearn.model_selection import cross_val_score
from tqdm import tqdm 
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

In [2]:
#=============SETTINGS============
RAW_DATA_DIR = "../data/raw"
TASK_PARAGIDM="left_right_hand"

# Utils

In [3]:
#Run Type to Task Map
run_type_to_task = {
    "R01": {
        "name": "Baseline - Eyes Open",
        "task_type": "baseline",
        "labels": None
    },
    "R02": {
        "name": "Baseline - Eyes Closed",
        "task_type": "baseline",
        "labels": None
    },
    "R03": {
        "name": "Task 1 - Real Left/Right Fist",
        "task_type": "motor_execution",
        "paradigm": "left_right_hand",
        "labels": {
            "T1": "left_fist",
            "T2": "right_fist"
        }
    },
    "R04": {
        "name": "Task 2 - Imagine Left/Right Fist",
        "task_type": "motor_imagery",
        "paradigm": "left_right_hand",
        "labels": {
            "T1": "left_fist",
            "T2": "right_fist"
        }
    },
    "R05": {
        "name": "Task 3 - Real Fists/Feet",
        "task_type": "motor_execution",
        "paradigm": "hands_feet",
        "labels": {
            "T1": "both_fists",
            "T2": "both_feet"
        }
    },
    "R06": {
        "name": "Task 4 - Imagine Fists/Feet",
        "task_type": "motor_imagery",
        "paradigm": "hands_feet",
        "labels": {
            "T1": "both_fists",
            "T2": "both_feet"
        }
    },
    "R07": {
        "name": "Task 1 - Real Left/Right Fist",
        "task_type": "motor_execution",
        "paradigm": "left_right_hand",
        "labels": {
            "T1": "left_fist",
            "T2": "right_fist"
        }
    },
    "R08": {
        "name": "Task 2 - Imagine Left/Right Fist",
        "task_type": "motor_imagery",
        "paradigm": "left_right_hand",
        "labels": {
            "T1": "left_fist",
            "T2": "right_fist"
        }
    },
    "R09": {
        "name": "Task 3 - Real Fists/Feet",
        "task_type": "motor_execution",
        "paradigm": "hands_feet",
        "labels": {
            "T1": "both_fists",
            "T2": "both_feet"
        }
    },
    "R10": {
        "name": "Task 4 - Imagine Fists/Feet",
        "task_type": "motor_imagery",
        "paradigm": "hands_feet",
        "labels": {
            "T1": "both_fists",
            "T2": "both_feet"
        }
    },
    "R11": {
        "name": "Task 1 - Real Left/Right Fist",
        "task_type": "motor_execution",
        "paradigm": "left_right_hand",
        "labels": {
            "T1": "left_fist",
            "T2": "right_fist"
        }
    },
    "R12": {
        "name": "Task 2 - Imagine Left/Right Fist",
        "task_type": "motor_imagery",
        "paradigm": "left_right_hand",
        "labels": {
            "T1": "left_fist",
            "T2": "right_fist"
        }
    },
    "R13": {
        "name": "Task 3 - Real Fists/Feet",
        "task_type": "motor_execution",
        "paradigm": "hands_feet",
        "labels": {
            "T1": "both_fists",
            "T2": "both_feet"
        }
    },
    "R14": {
        "name": "Task 4 - Imagine Fists/Feet",
        "task_type": "motor_imagery",
        "paradigm": "hands_feet",
        "labels": {
            "T1": "both_fists",
            "T2": "both_feet"
        }
    }
}

MOTOR_CHANNELS = [
    'C3..',   # Left motor cortex (primary)
    'Cz..',   # Central motor area (feet)
    'C4..',   # Right motor cortex (primary)
    'Fc3.',   # Left frontal-central (premotor)
    'Fc4.',   # Right frontal-central (premotor)
    'Cp3.',   # Left central-parietal (sensorimotor)
    'Cp4.',   # Right central-parietal (sensorimotor)
    'C5..',   # Left lateral motor
    'C1..',   # Left medial motor
    'C2..',   # Right medial motor
    'C6..',   # Right lateral motor
    'Fc1.',   # Left medial frontal-central
    'Fc2.',   # Right medial frontal-central
    'Fc5.',   # Left lateral frontal-central
    'Fc6.',   # Right lateral frontal-central
    'Cp1.',   # Left medial central-parietal
    'Cp2.',   # Right medial central-parietal
    'Cp5.',   # Left lateral central-parietal
    'Cp6.'    # Right lateral central-parietal
]

extract_task_id = lambda filepath: 'R' + filepath.split('R')[-1].split('.')[0]

def rename_annotations(raw, run_type):
    """
    Rename MNE annotations to readable task labels
    """
    task_info = run_type_to_task[run_type]
    
    if task_info['labels'] is not None:
        # Map T0, T1, T2 to readable names
        annotation_mapping = {
            'T0': 'rest',
            'T1': task_info['labels']['T1'],
            'T2': task_info['labels']['T2']
        }
        
        raw.annotations.rename(annotation_mapping)
    
    # Extract events with new names
    events, event_dict = mne.events_from_annotations(raw)
    
    return event_dict

def select_eeg_files_for_subject_by_paradigm(subject_id, paradigm, task_type="motor_imagery"):
    """
    Select motor imagery EEG files for a subject by paradigm
    """
    matching_runs = [
        run_id for run_id, info in run_type_to_task.items()
        if info.get('paradigm') == paradigm and info['task_type'] == task_type
    ]
    
    return [
        os.path.join(RAW_DATA_DIR, subject_id, f"{subject_id}{run_id}.edf")
        for run_id in sorted(matching_runs)
    ]


def load_and_concatenate_to_epochs(files, tmin=-0.5, tmax=3.5, picks=MOTOR_CHANNELS):
    """
    Load multiple EDF files and concatenate into single Epochs object
    """
    epochs_list = []
    
    for filepath in files:
        # Extract run_id
        run_id = extract_task_id(filepath)
        
        # Load raw
        raw = mne.io.read_raw_edf(filepath, preload=True, verbose=False)
        
        # Rename annotations
        event_dict = rename_annotations(raw, run_id)
        
        # Get events (exclude rest)
        events, _ = mne.events_from_annotations(raw)
        event_id = {k: v for k, v in event_dict.items() if k != 'rest'}
        
        # Create epochs
        epochs = mne.Epochs(
            raw, events, event_id=event_id,
            tmin=tmin, tmax=tmax,
            baseline=None,
            picks=picks,
            preload=True,
            verbose=False
        )
        
        epochs_list.append(epochs)
    
    # Concatenate
    return mne.concatenate_epochs(epochs_list)



# Classification Pipeline + Wavelets (Bonus)

## Haar Wavelets

In [4]:
def haar_dwt_forward(signal):
    """One-level DWT - vectorized"""
    N = len(signal)
    if N % 2:
        signal = np.append(signal, signal[-1])
        N = len(signal)
    
    # Reshape to pairs: (N//2, 2)
    pairs = signal.reshape(N//2, 2)
    
    # Vectorized operations
    approx = (pairs[:, 0] + pairs[:, 1]) / np.sqrt(2)
    detail = (pairs[:, 0] - pairs[:, 1]) / np.sqrt(2)
    
    return approx, detail

def haar_dwt_multilevel(signal, n_levels=5):
    """
    Multi-level Haar DWT - vectorized.
    
    Returns: [approximation, detail_n, detail_n-1, ..., detail_1]
    """
    coeffs = []
    current = signal.copy()
    
    for _ in range(n_levels):
        if len(current) < 2:
            break
        approx, detail = haar_dwt_forward(current)
        coeffs.append(detail)
        current = approx
    
    coeffs.append(current)
    return coeffs[::-1]


In [5]:
from sklearn.base import BaseEstimator, TransformerMixin

class HaarWaveletTransform(BaseEstimator, TransformerMixin):
    """Apply Haar to each channel, keep 3D shape for CSP."""
    def __init__(self, n_levels=4):
        self.n_levels = n_levels
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        """
        X: (n_trials, n_channels, n_times)
        Returns: (n_trials, n_channels, n_wavelet_coeffs)
        """
        n_trials, n_channels, n_times = X.shape
        
        # Apply DWT to get coefficient length
        sample_coeffs = haar_dwt_multilevel(X[0, 0, :], n_levels=self.n_levels)
        n_coeffs = sum(len(c) for c in sample_coeffs)
        
        X_wavelet = np.zeros((n_trials, n_channels, n_coeffs))
        
        for trial in range(n_trials):
            for ch in range(n_channels):
                coeffs = haar_dwt_multilevel(X[trial, ch, :], n_levels=self.n_levels)
                # Flatten coefficients
                X_wavelet[trial, ch, :] = np.concatenate(coeffs)
        
        # Replace any inf/nan
        X_wavelet = np.nan_to_num(X_wavelet, nan=0.0, posinf=1e10, neginf=-1e10)
        
        return X_wavelet


### CSP Implementation



In [6]:
from sklearn.base import BaseEstimator, TransformerMixin
import numpy as np
from scipy import linalg

class CommonSpatialPattern(BaseEstimator, TransformerMixin):
    def __init__(self, n_components):

        if n_components % 2:
            raise ValueError(f"n_components should be even number! Got " + n_components)

        self.n_components  = n_components
    
    def _compute_average_covariance_matrix(self, X_class):
        """Compute average normalized covariance matrix for one class"""

        n_trials, n_channels, n_times = X_class.shape

        cov_sum = np.zeros((n_channels, n_channels))

        for i in range(n_trials):
            trial = X_class[i]

            #computes covariance matrix
            cov = np.cov(trial)

            #normalize by trace (sum of diagonal elements)
            cov = cov / np.trace(cov)
            cov_sum += cov
        
        cov_avg = cov_sum / n_trials
        return cov_avg

            
    def fit(self, X, y):
        
        #check only 2 classes present
        classes = np.unique(y)
        assert len(classes) == 2

        X_class0 = X[y == classes[0]]
        X_class1 = X[y == classes[1]]

        #compute Covariance matrices for both
        cov0 = self._compute_average_covariance_matrix(X_class=X_class0)
        cov1 = self._compute_average_covariance_matrix(X_class=X_class1)

        # Solve generalized eigenvalue problem
        # Find W such that: cov_class0 @ W = λ @ cov_class1 @ W
        eigenvalues, eigenvectors = linalg.eigh(cov0, cov1)

        #Now, we sort by eignenvalues in desc order
        sorted_idx = np.argsort(eigenvalues)[::-1]
        eigenvalues = eigenvalues[sorted_idx]
        eigenvectors = eigenvectors[:, sorted_idx]

        # Now, components at the begining of list are such that var for class0 is minimized while var for class1 is maximized.
        # At the end of list, we have the opposite.
        # We need to select  n_components/2 from top and bottom

        selected_components = np.concatenate(
            [
            eigenvectors[:, :self.n_components // 2], #top
            eigenvectors[:, - self.n_components // 2:], #bottom
            ],
            axis=1
        )

        self.filters_ = selected_components # (n_channels, n_components)
        self.patterns_ = linalg.pinv(self.filters_.T) #inverse for interpretation

        return self

    def transform(self, X):
        """Applies learnt CSP filters and transform trial from (n_channels, n_times) to (n_components,)"""

        n_trials = X.shape[0]

        if self.filters_ is None:
            raise ValueError("Must call fit() before transform()!")

        X_transformed = np.zeros((n_trials, self.n_components))

        
        for i in range(n_trials):

            filtered = self.filters_.T @ X[i] #produces (n_components, n_times) shape

            #collapse time dimension by computing variance
            features = np.log(np.var(filtered, axis=1))
            X_transformed[i, :] = features
            

        return X_transformed



## Pipeline

In [7]:
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
#from mne.decoding import CSP
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from tqdm import tqdm

mne.set_log_level('WARNING')

#HYPERPARAMS
TEST_SIZE=0.20
K_FOLDS=5
N_COMPONENTS=6
WAVELET_LEVEL=8
RANDOM_STATE=42
L_FREQ=8
H_FREQ=30
TMIN=0.5
TMAX=3
TASK_TYPE="motor_execution"
LIMIT=109

def get_epochs_for_subject(subject_id, paradigm, task_type):
    files = select_eeg_files_for_subject_by_paradigm(subject_id, paradigm, task_type)
    epochs = load_and_concatenate_to_epochs(files, tmin=TMIN, tmax=TMAX, picks=MOTOR_CHANNELS)
    epochs.filter(l_freq=L_FREQ, h_freq=H_FREQ, verbose=False) #Filter
    return epochs


def get_pipeline_scores_for_subject(subject_id, paradigm, task_type):

    epochs = get_epochs_for_subject(subject_id, paradigm, task_type)
    
    # Extract data and labels
    X = epochs.get_data()  # (n_trials, n_channels, n_times)
    y = epochs.events[:, -1]  # Labels (2 or 3 for left/right)
    
    # Train/test split
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=TEST_SIZE, random_state=RANDOM_STATE, stratify=y
    )
    
    # Create pipeline
    pipeline = Pipeline([
        ('wavelet', HaarWaveletTransform(n_levels=WAVELET_LEVEL)),
        ('csp', CommonSpatialPattern(n_components=N_COMPONENTS)),
        #('csp', CSP(n_components=N_COMPONENTS, reg=None, log=True)),
        ('lda', LinearDiscriminantAnalysis())
    ])
    
    # Cross-validation on training set
    cv_scores = cross_val_score(pipeline, X_train, y_train, cv=K_FOLDS)
    
    # Train on full training set and evaluate on test set
    pipeline.fit(X_train, y_train)
    test_score = pipeline.score(X_test, y_test)
    
    return {
        'cv_score': cv_scores.mean(),
        'cv_std': cv_scores.std(),
        'test_score': test_score
    }



subject_ids = [f"S{i:03d}" for i in range(1, 110)]


In [8]:
def evaluate_pipeline_on_experiment(task_paradigm, task_type):
    
    subject_results = []
    
    
    for i in tqdm(range(LIMIT)):
        results = get_pipeline_scores_for_subject(subject_ids[i], task_paradigm, task_type)
        subject_results.append(results)

    
    results_df = pd.DataFrame(subject_results)
    return {
        "task_paradigm" : task_paradigm,
        "task_type" : task_type,
        "cv_score" : float(results_df.mean()["cv_score"]),
        "test_score" : float(results_df.mean()["test_score"])
    }


    

## Evaluate on all experiments


In [9]:
# Run 4 experiments - print each as it completes
experiments = []

exp0 = evaluate_pipeline_on_experiment('left_right_hand', 'motor_execution')
print(f"experiment 0: accuracy = {exp0['test_score']:.4f}")
experiments.append(exp0)

exp1 = evaluate_pipeline_on_experiment('left_right_hand', 'motor_imagery')
print(f"experiment 1: accuracy = {exp1['test_score']:.4f}")
experiments.append(exp1)

exp2 = evaluate_pipeline_on_experiment('hands_feet', 'motor_execution')
print(f"experiment 2: accuracy = {exp2['test_score']:.4f}")
experiments.append(exp2)

exp3 = evaluate_pipeline_on_experiment('hands_feet', 'motor_imagery')
print(f"experiment 3: accuracy = {exp3['test_score']:.4f}")
experiments.append(exp3)

# Final mean
mean_accuracy = sum(e['test_score'] for e in experiments) / 4
print(f"\nMean accuracy of 4 experiments: {mean_accuracy:.4f}")

 13%|█████▍                                    | 14/109 [00:04<00:27,  3.47it/s]


KeyboardInterrupt: 